In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
csv_file="movie_dataset.csv"
df=pd.read_csv(csv_file)
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [4]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
conn=sqlite3.connect(':memory:')
df.to_sql("movie",conn,index=True)

4803

In [6]:
# Example SQL query to select data
query = "SELECT * FROM movie"
result = pd.read_sql_query(query, conn)
print(result)


      level_0  index     budget                                    genres  \
0           0      0  237000000  Action Adventure Fantasy Science Fiction   
1           1      1  300000000                  Adventure Fantasy Action   
2           2      2  245000000                    Action Adventure Crime   
3           3      3  250000000               Action Crime Drama Thriller   
4           4      4  260000000          Action Adventure Science Fiction   
...       ...    ...        ...                                       ...   
4798     4798   4798     220000                     Action Crime Thriller   
4799     4799   4799       9000                            Comedy Romance   
4800     4800   4800          0             Comedy Drama Romance TV Movie   
4801     4801   4801          0                                      None   
4802     4802   4802          0                               Documentary   

                                               homepage      id  \
0       

In [7]:
#1. Which is the movie(s) with the smallest runtime? Which is the movie(s) with the highest runtime?
query1="""SELECT 
         title,runtime FROM movie WHERE runtime!="None" ORDER BY runtime LIMIT 1"""
result = pd.read_sql_query(query1, conn)
print(result)

             title  runtime
0  The Tooth Fairy      0.0


In [8]:
query2="""SELECT 
         title,runtime FROM movie WHERE runtime!="None" ORDER BY runtime DESC LIMIT 1"""
result = pd.read_sql_query(query2, conn)
print(result)

    title  runtime
0  Carlos    338.0


In [9]:
# 2. Take the top 5 production houses (by budget) and list their top 5 most popular movies, their revenue and vote_average.

query3="""
SELECT
title,
popularity,
revenue,
vote_average,
production_companies
FROM movie WHERE production_companies IN
(
SELECT production_companies 
FROM movie
GROUP BY production_companies
ORDER BY SUM(budget) DESC
LIMIT 5
) AND title IN (
SELECT
title
FROM
(
SELECT
title,
DENSE_RANK() OVER(PARTITION BY production_companies ORDER BY popularity DESC) AS r
FROM
movie
)
WHERE
r <=5
)
ORDER BY production_companies

"""
result = pd.read_sql_query(query3, conn)
print(result)


                      title  popularity     revenue  vote_average  \
0                   Skyfall   93.004993  1108561013           6.9   
1          Charlie's Angels   40.203950   264105545           5.6   
2          Captain Phillips   52.786917    95000000           7.6   
3                 Big Daddy   34.868590   234801895           6.1   
4             Groundhog Day   52.744331    70906973           7.4   
5     Rise of the Guardians   61.788035   306941670           7.1   
6    Penguins of Madagascar   84.366984   373552094           6.5   
7             Kung Fu Panda   84.689648   631744560           6.9   
8                  Megamind   68.757242   321887208           6.7   
9                The Croods   64.183321   585178928           6.8   
10               Iron Man 3   77.682080  1215439994           6.8   
11                 Iron Man  120.725053   585174222           7.4   
12     Thor: The Dark World   99.499595   644571402           6.8   
13                     Thor   86.4

In [10]:
#3. List the production house for every year from 2000-2016 which has released the most number of movies in that year.
query4="""
SELECT
production_companies,
COUNT(title) AS count_of_movies
FROM movie
GROUP BY production_companies
ORDER BY count_of_movies DESC
"""

In [11]:
result = pd.read_sql_query(query4,conn)
print(result)


                                   production_companies  count_of_movies
0                                                    []              351
1             [{"name": "Paramount Pictures", "id": 4}]               58
2            [{"name": "Universal Pictures", "id": 33}]               45
3               [{"name": "New Line Cinema", "id": 12}]               38
4              [{"name": "Columbia Pictures", "id": 5}]               37
...                                                 ...              ...
3692  [{"name": "1492 Pictures", "id": 436}, {"name"...                1
3693  [{"name": "1492 Pictures", "id": 436}, {"name"...                1
3694      [{"name": "101st Street Films", "id": 54684}]                1
3695  [{"name": "100 Bares", "id": 5517}, {"name": "...                1
3696  [{"name": "100 Bares", "id": 5517}, {"name": "...                1

[3697 rows x 2 columns]


#4. You are going to invest all your life's savings in a production company. You have two choices: "Marvel Studios" and "DC Comics". Which company would you bet on? This is an open ended question. Define your own metrics to measure which one is a better investment opportunity and defend your analysis.

In [12]:
query5="""
SELECT title,
budget,
revenue,
(revenue-budget) AS profit,
(revenue/budget)*100 AS profit_percentage
FROM movie
WHERE json_extract(production_companies, '$[0].name') = "DC Comics"
ORDER BY release_date
"""

In [13]:
result2= pd.read_sql_query(query5,conn)
print(result2)


                                     title     budget     revenue     profit  \
0                            Batman Begins  150000000   374218673  224218673   
1                         Superman Returns  270000000   391081192  121081192   
2                          The Dark Knight  185000000  1004558444  819558444   
3                               The Spirit   60000000    39031337  -20968663   
4                               The Losers   25000000    23580000   -1420000   
5                                      RED   58000000    71664962   13664962   
6  Batman: The Dark Knight Returns, Part 2    3500000           0   -3500000   
7                                    RED 2   84000000           0  -84000000   
8       Batman v Superman: Dawn of Justice  250000000   873260194  623260194   
9                            Suicide Squad  175000000   745000000  570000000   

   profit_percentage  
0                200  
1                100  
2                500  
3                  0  
4   

In [14]:
query6="""
SELECT title,
budget,
revenue,
(revenue-budget) AS profit,
(revenue/budget)*100 AS profit_percentage
FROM movie
WHERE json_extract(production_companies, '$[0].name') = "Marvel Studios"
ORDER BY release_date
"""

In [15]:
result3= pd.read_sql_query(query6,conn)
print(result3)


                                 title     budget     revenue      profit  \
0                             Iron Man  140000000   585174222   445174222   
1                           Iron Man 2  200000000   623933331   423933331   
2                                 Thor  150000000   449326618   299326618   
3   Captain America: The First Avenger  140000000   370569774   230569774   
4                           Iron Man 3  200000000  1215439994  1015439994   
5                 Thor: The Dark World  170000000   644571402   474571402   
6  Captain America: The Winter Soldier  170000000   714766572   544766572   
7              Guardians of the Galaxy  170000000   773328629   603328629   
8              Avengers: Age of Ultron  280000000  1405403694  1125403694   
9                              Ant-Man  130000000   519311965   389311965   

   profit_percentage  
0                400  
1                300  
2                200  
3                200  
4                600  
5             

In [21]:
"""FROM ABOVE ANALYSIS
There are 10 movies relased by "Marvel Studios"
and 10 movies relased by "DC Comics"
I calculated profit and profit percentage from given 
dataset .
I found movie such as "The Spirit", "The Losers", etc 
produced by  DC Comics had high budget but very low revenue.
Despite,having high budget, Marvel Studios has high revenue

So i will choose "Marvel Studios" for investment

"""
increase_in_revenue=(result3["revenue"].aggregate(["sum"])-result2["revenue"].aggregate(["sum"]))/result3["revenue"].aggregate(["sum"])

In [23]:
increase_in_revenue*100

sum    51.760084
Name: revenue, dtype: float64